In [1]:
using DataFrames, Distributions, DataFramesMeta

In [2]:
function gen_data(
    ; 
    n = 500000, 
    mu = [0, 2], 
    sigma = 1, 
    pZ1 = 0.8)
    
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [3]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.732765,0.30501,0.17873
2,2,3.53187,0.000780166,0.123408
3,3,-1.04659,0.230705,0.00384956
4,4,-0.514653,0.349458,0.016896
5,5,2.4188,0.0214029,0.365447
6,6,0.0112174,0.398917,0.0552125


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.00001,
    max_iterations = 1000,
    verbose = false
    )

    # pre-allocate a table of results by iteration:
    progress = DataFrame(
        Iteration = 0:max_iterations, 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        loglik = Vector{Float64}(undef, max_iterations+1), 
        diff_loglik = Vector{Float64}(undef, max_iterations+1)
        )

    # initial E step, to perform needed calculations for initial likelihood:
    E_step!(data, pi_hat_0) 
    ll = loglik(data)
    progress[1, :] = (0, pi_hat_0, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations

        # M step: re-estimate parameters
        pi_hat = M_step(data)

        # E step: re-compute distribution of missing variables, using parameters
        E_step!(data, pi_hat)
        
        # Assess convergence

        ## save the previous log-likelihood so we can test for convergence
        ll_old = ll
        
        ## here's the new log-likelihood
        ll = loglik(data)

        ll_diff = ll - ll_old
        
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)
        
        if(verbose)
            print(progress[i+1,:])
        end
        
        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    data[:, :pZ1_given_Y] |> mean
end

M_step (generic function with 1 method)

In [7]:
function loglik(data)
    data[:, :pY] .|> log |> sum
end

loglik (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = 0.00001);


  1.367570 seconds (6.98 M allocations: 881.487 MiB, 10.25% gc time, 87.37% compilation time)


The first run requires compilation; on subsequent calls it will be faster:

In [9]:
@time progress = fit_model!(data, tolerance = 0.00001)

  0.148797 seconds (13.08 k allocations: 504.204 MiB, 16.27% gc time)


,Iteration,pi_hat,loglik,diff_loglik
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77315e5,NaN
2,1,0.664833,-8.36983e5,40332.0
3,2,0.737006,-8.2805e5,8932.56
4,3,0.76945,-8.25977e5,2072.58
5,4,0.784811,-8.25472e5,505.626
6,5,0.792342,-8.25344e5,127.287
7,6,0.796107,-8.25312e5,32.6051
8,7,0.79801,-8.25303e5,8.42914
9,8,0.798976,-8.25301e5,2.18957
